In [ ]:
import numpy as np

#Eigendecomposition is used. U = QAQ^(-1)
#where A is a diagonal matrix whose entries are the eigenvalues of U,
#and Q is a matrix whose columns are the eigenvectors of U

#Procedure goes as follows:
#1) Generate random Q matrix
#2) Check is invertible. If not generate new random Q matrix
#3) Assuming Q is invertible, generate diagonales entries of A 
#   such that they have the needed properties (e.g. eigenvalues>0)
#4) Calculate U


#Parameters for Q
N = 4      #dimension of random square matrix
scale = 1 #random matrix by fault between [0,1), reason why it should be scaled

#Generate nonsingunlar Q (invertible)
valid = False
while True:
    Q     = scale*np.random.rand(N,N)
    try:
        inv_Q = np.linalg.inv(Q)
        break
    except np.linalg.LinAlgError:
        print("sampled Q matrix is not invertible")


#Generate eigenvalues for A
lb = -1 #lower bound for eigenvalues
ub = 1  #upper bound for eigenvalues
eigval = (ub-lb)*np.random.random(N) + lb #numpy function samples from U(0,1)
A = np.diag(eigval)

#Calculate desired matrix U and verify correct eigenvalues
U = Q @ A @ inv_Q#np.linalg.inv(Q)
print(eigval)
print(np.linalg.eigvals(U))

# Trabajo pendiente
1) Programar maximum likelihood (optimizacion)
2) Calcular periodic trend

In [ ]:
from scipy.stats import uniform
from scipy.stats import multivariate_normal
from scipy.stats import norm
from scipy.stats import truncnorm
import numpy as np

#DISCLAIMER: CODED FOR VAR OF ORDER 1

## Previous values
A_old = np.array([[ 0.19342949,0.06497526,0.20640138],
 [ 0.71551897,-0.8875307,0.81804304],
 [ 0.55257784,-0.47167893,0.54647069]])
samp_vecA_old = np.array([[-0.61605662,1.60194096,1.54104974],
 [ 3.0063165,1.95457467,-0.34768391],
 [ 1.49965918,1.98081968,-1.70410413]])
samp_valA_old = np.array([[-0.62608599,0.      ,0.        ],
 [ 0.       ,0.52792554,0.        ],
 [ 0.       ,0.     ,-0.04947007]])


U_old = np.array([[ 0.2033993,-0.18571928,0.33787476],
 [ 0.19604154,-1.15594823,2.57610726],
 [ 0.10936385,-0.74142332,1.61134019]])
samp_vecU_old = np.array([[ 0.04359329,1.03759474,-1.67779624],
 [ 0.97607584,0.64018012,1.39118497],
 [ 0.54931646,0.25065424,0.8522514 ]])
samp_valU_old = np.array([[0.30259028,0.   ,0. ],
 [0.    ,0.17043456,0.  ],
 [0.    ,0.     ,0.18576643]])

#theta_old = np.concatenate((A_old.reshape(-1),U_old.reshape(-1)))
theta_old = np.concatenate((samp_vecA_old.reshape(-1),samp_vecU_old.reshape(-1),
                            np.diag(samp_valA_old),np.diag(samp_valU_old)))

## User Parameters
K =  3 #problem dimension(number of wind farms)
j =  1 #index which is left unconditioned of vector theta, j = 1,...,K(K+1)
delta = 0.0001 #avoid exactly taking bounds
user_std = 1

covA = np.eye(K*K)*0.0001 #covariance matrix for eigenvectors of A (K x K)
covU = np.eye(K*K)*0.0001 #covariance matrix for eigenvectors of U (K x K)

## Processing of previous values
muA = samp_vecA_old.reshape(-1) #for eigenvector matrix of A
muU = samp_vecU_old.reshape(-1) #for eigenvector matrix of U

## Jumping distribution of theta, conditioned on all values except index j
#mu = theta_old[j]
#if (j < (K*K)):
#    rv = norm(loc=mu,scale=user_std)
#elif ( (j >= (K*K)) and (j < (K*K+K)) ):
#    a, b = (myclip_a - my_mean) / my_std, (myclip_b - my_mean) / my_std
#    rv = truncnorm(a=a,b=b,loc=,scale=)
#else:
#    print("ERROR: index j out of bounds")


## Jumping distributions for A1,U (theta)
#both matrices are computed by eigendecomposition to ensure eigenvalues are within certain range
#code is programmed just for VAR of order 1, i.e., A = A1

#jumping distribution for eigenvectors of A1
#muA  = np.zeros((K*K,)) 
rv_gaussA = multivariate_normal(mean=muA,cov=covA)

#jumping distribution for eigenvalues of A1
lbA = -1 + delta
ubA = 1 - delta
rv_unifA = uniform(loc=lbA,scale=ubA-lbA) #support in [loc,loc+scale]

#jumping distribution for eigenvectors of U (covU)
#muU  = np.zeros((K*K,))
rv_gaussU = multivariate_normal(mean=muU,cov=covU)

#jumping distribution for eigenvalues of U (covU)
lbU = 0 + delta
ubU = 1 - delta
rv_unifU = uniform(loc=lbU,scale=ubU-lbU)

## Samples are generated
samp_vecA = rv_gaussA.rvs()
samp_valA = rv_unifA.rvs(size=K)
samp_vecU = rv_gaussU.rvs()
samp_valU = rv_unifU.rvs(size=K)

## Resulting A,U are computed and transformed to vector theta
samp_vecA = np.reshape(samp_vecA,(K,K))
samp_valA = np.diag(samp_valA)
samp_vecU = np.reshape(samp_vecU,(K,K))
samp_valU = np.diag(samp_valU)

A = samp_vecA @ samp_valA @np.linalg.inv(samp_vecA)
U = samp_vecU @ samp_valU @np.linalg.inv(samp_vecU)

theta = np.concatenate((samp_vecA.reshape(-1),samp_vecU.reshape(-1),np.diag(samp_valA),np.diag(samp_valU)))
#theta = np.concatenate((A.reshape(-1),U.reshape(-1)))

print(samp_vecA)
print(samp_vecU)
print(samp_valA)
print(samp_valU)

In [ ]:
#print(A)
#print(samp_vecA)
#print(samp_valA)
#print(U)
#print(samp_vecU)
#print(samp_valU)

#pv = 1
#Kv = 3
#A_test    = np.reshape(theta[:pv*Kv**2],(Kv*pv,Kv)) ##OJO YA NO ES NECESARIO SWAPAXES
#CovU_test = np.reshape(theta[pv*Kv**2:],(Kv,Kv))